# Module 10: Final Trading Project - Complete Trading System

**Difficulty**: ⭐⭐⭐ (Advanced)  
**Estimated Time**: 120 minutes  
**Prerequisites**: 
- [Module 00: Setup and Introduction](00_setup_introduction.ipynb)
- [Module 01: Introduction to Technical Analysis](01_introduction_to_technical_analysis.ipynb)
- [Module 02: Understanding Price Charts](02_understanding_price_charts.ipynb)
- [Module 03: Moving Averages and Trend Analysis](03_moving_averages_and_trend_analysis.ipynb)
- [Module 04: RSI and MACD Oscillators](04_rsi_and_macd.ipynb)
- [Module 05: Chart Patterns Recognition](05_chart_patterns.ipynb)
- [Module 06: Volume Analysis](06_volume_analysis.ipynb) (assumed)
- [Module 07: Support and Resistance](07_support_resistance.ipynb) (assumed)
- [Module 08: Risk Management](08_risk_management.ipynb) (assumed)
- [Module 09: Backtesting Strategies](09_backtesting.ipynb) (assumed)

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Design and implement** a complete stock screening system for FBM KLCI constituents
2. **Create multi-indicator entry signals** combining moving averages, RSI, MACD, volume, and patterns
3. **Calculate position sizes** using the 2% risk rule with ATR-based stop-loss placement
4. **Build a trade management system** with dynamic stop-loss, take-profit, and trailing stops
5. **Backtest strategies** across multiple Malaysian stocks simultaneously
6. **Analyze performance metrics** comprehensively (returns, Sharpe ratio, drawdown, win rate)
7. **Compare strategy performance** against FBM KLCI benchmark
8. **Create risk management dashboards** for portfolio monitoring
9. **Calculate realistic transaction costs** for Malaysian market (brokerage, clearing, stamp duty)
10. **Transition to paper trading** with confidence using systematic approach

---

## Introduction: Your Complete Trading System

**Congratulations on reaching the final module!** 🎓

You've learned individual technical analysis components across 9 modules. Now it's time to integrate everything into a **complete, professional trading system** that you can actually use.

### What Makes a Complete Trading System?

A professional trading system requires:

1. **Stock Screening**: Identify opportunities from universe of stocks
2. **Entry Signals**: Multiple confirmation indicators (not just one!)
3. **Position Sizing**: Calculate how much to invest per trade
4. **Risk Management**: Stop-loss, take-profit, maximum portfolio exposure
5. **Trade Management**: Adjust positions as market moves
6. **Performance Tracking**: Monitor what works and what doesn't
7. **Portfolio Management**: Handle multiple positions simultaneously

### Real-World Trading in Malaysia

This module uses:
- **Real Malaysian stocks** from multiple sectors (banking, construction, plantation)
- **Realistic transaction costs** (0.1% brokerage + 0.03% clearing + 0.1% stamp duty)
- **Practical position sizing** suitable for retail accounts (RM 10,000 - RM 100,000)
- **FBM KLCI benchmark** for performance comparison

### Learning Approach

We'll build the system progressively:
1. Create each component as reusable function
2. Test each component individually
3. Integrate into complete system
4. Backtest on historical data
5. Analyze results and refine

**Remember**: This is educational software. Always paper trade first before risking real money!

---

## Setup and Configuration

Import all required libraries and set up configuration parameters.

In [ ]:
# Standard libraries
import warnings
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Data fetching
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)

print("All libraries imported successfully!")
print(f"Setup completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Trading System Configuration

Define global parameters for the trading system.

In [ ]:
# Malaysian stock universe (diverse sectors)
STOCK_UNIVERSE = {
    # Banking sector
    '1155.KL': 'Maybank',
    '1023.KL': 'CIMB Group',
    '1295.KL': 'Public Bank',
    
    # Construction & Infrastructure
    '5398.KL': 'Gamuda',
    '3336.KL': 'IJM Corp',
    
    # Plantation
    '5285.KL': 'Sime Darby Plantation',
    '1961.KL': 'IOI Corporation',
    
    # Technology
    '0166.KL': 'Inari Amertron',
    
    # Healthcare/Gloves
    '5225.KL': 'Top Glove',
    '5168.KL': 'Hartalega',
}

# Benchmark index
BENCHMARK_SYMBOL = '^KLSE'  # FBM KLCI

# Data period for backtesting
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'

# Malaysian transaction costs (per trade)
TRANSACTION_COSTS = {
    'brokerage': 0.001,      # 0.1% (typical online broker)
    'clearing_fee': 0.0003,  # 0.03%
    'stamp_duty': 0.001,     # 0.1% (RM1 per RM1000)
}
TOTAL_COST_PER_TRADE = sum(TRANSACTION_COSTS.values())  # 0.23%

# Portfolio parameters
INITIAL_CAPITAL = 100000  # RM 100,000
MAX_RISK_PER_TRADE = 0.02  # 2% maximum risk per trade
MAX_POSITIONS = 5  # Maximum simultaneous positions
MAX_PORTFOLIO_RISK = 0.06  # 6% maximum total portfolio risk

# Technical indicator parameters
PARAMS = {
    'ma_short': 20,
    'ma_long': 50,
    'rsi_period': 14,
    'rsi_oversold': 30,
    'rsi_overbought': 70,
    'macd_fast': 12,
    'macd_slow': 26,
    'macd_signal': 9,
    'atr_period': 14,
    'volume_ma': 20,
}

# Trade management
STOP_LOSS_ATR_MULTIPLIER = 2.0  # Stop loss = 2 x ATR below entry
TAKE_PROFIT_ATR_MULTIPLIER = 3.0  # Take profit = 3 x ATR above entry
TRAILING_STOP_ATR_MULTIPLIER = 1.5  # Trailing stop activates at 1.5 x ATR profit

print("Trading System Configuration")
print("=" * 60)
print(f"Stock Universe: {len(STOCK_UNIVERSE)} Malaysian stocks")
print(f"Backtest Period: {START_DATE} to {END_DATE}")
print(f"Initial Capital: RM {INITIAL_CAPITAL:,.0f}")
print(f"Transaction Cost: {TOTAL_COST_PER_TRADE*100:.2f}% per trade")
print(f"Max Risk Per Trade: {MAX_RISK_PER_TRADE*100:.1f}%")
print(f"Max Positions: {MAX_POSITIONS}")
print(f"Max Portfolio Risk: {MAX_PORTFOLIO_RISK*100:.1f}%")

---

## Part 1: Stock Screening System

First component: scan the stock universe to identify potential trading opportunities.

### Screening Criteria

A stock passes screening if:
1. **Trend**: Price above 50-day MA (uptrend)
2. **Momentum**: 20-day MA above 50-day MA (bullish crossover region)
3. **Volume**: Recent volume above 20-day average (interest/activity)
4. **Volatility**: ATR between reasonable bounds (not too choppy, not too flat)
5. **Data quality**: Sufficient historical data available

In [ ]:
def download_stock_data(
    symbol: str, 
    start_date: str, 
    end_date: str
) -> Optional[pd.DataFrame]:
    """
    Download historical stock data from Yahoo Finance.
    
    Returns None if download fails or insufficient data.
    """
    try:
        data = yf.download(
            symbol, 
            start=start_date, 
            end=end_date, 
            progress=False,
            auto_adjust=True  # Adjust for splits and dividends
        )
        
        # Validate data quality
        if data.empty or len(data) < 100:
            return None
            
        # Rename columns to standard format
        data.columns = [col.lower() for col in data.columns]
        
        return data
        
    except Exception as e:
        print(f"Error downloading {symbol}: {e}")
        return None


def calculate_indicators(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate all technical indicators needed for screening and trading.
    
    Adds columns for: MAs, RSI, MACD, ATR, volume indicators.
    """
    df = data.copy()
    
    # Moving Averages
    df['ma_short'] = df['close'].rolling(window=PARAMS['ma_short']).mean()
    df['ma_long'] = df['close'].rolling(window=PARAMS['ma_long']).mean()
    
    # RSI (Relative Strength Index)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=PARAMS['rsi_period']).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=PARAMS['rsi_period']).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD (Moving Average Convergence Divergence)
    ema_fast = df['close'].ewm(span=PARAMS['macd_fast'], adjust=False).mean()
    ema_slow = df['close'].ewm(span=PARAMS['macd_slow'], adjust=False).mean()
    df['macd'] = ema_fast - ema_slow
    df['macd_signal'] = df['macd'].ewm(span=PARAMS['macd_signal'], adjust=False).mean()
    df['macd_histogram'] = df['macd'] - df['macd_signal']
    
    # ATR (Average True Range) for volatility
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = true_range.rolling(window=PARAMS['atr_period']).mean()
    
    # Volume indicators
    df['volume_ma'] = df['volume'].rolling(window=PARAMS['volume_ma']).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma']
    
    return df


def screen_stock(data: pd.DataFrame) -> Tuple[bool, Dict[str, any]]:
    """
    Screen a stock based on technical criteria.
    
    Returns:
        (passed, criteria_details)
    """
    # Get latest values
    latest = data.iloc[-1]
    
    # Check each criterion
    criteria = {
        'price_above_ma50': latest['close'] > latest['ma_long'],
        'ma20_above_ma50': latest['ma_short'] > latest['ma_long'],
        'volume_above_avg': latest['volume_ratio'] > 1.0,
        'atr_valid': 0.01 < (latest['atr'] / latest['close']) < 0.10,  # 1-10% volatility
        'rsi_not_extreme': PARAMS['rsi_oversold'] < latest['rsi'] < PARAMS['rsi_overbought'],
    }
    
    # Must pass all criteria
    passed = all(criteria.values())
    
    # Add current values for reference
    criteria['current_price'] = latest['close']
    criteria['current_rsi'] = latest['rsi']
    criteria['current_macd'] = latest['macd_histogram']
    
    return passed, criteria


# Test the screening system
print("Testing Stock Screening System...")
print("=" * 60)

# Download and screen one test stock
test_symbol = '1155.KL'
test_data = download_stock_data(test_symbol, START_DATE, END_DATE)

if test_data is not None:
    test_data = calculate_indicators(test_data)
    passed, criteria = screen_stock(test_data)
    
    print(f"\nTest Stock: {test_symbol} ({STOCK_UNIVERSE[test_symbol]})")
    print(f"Screening Result: {'✓ PASSED' if passed else '✗ FAILED'}")
    print("\nCriteria Details:")
    for criterion, value in criteria.items():
        if isinstance(value, bool):
            print(f"  {criterion}: {'✓' if value else '✗'}")
        else:
            print(f"  {criterion}: {value:.2f}")
else:
    print(f"Failed to download data for {test_symbol}")

### Screen the Entire Universe

Apply screening to all stocks in our universe.

In [ ]:
def screen_universe(
    universe: Dict[str, str],
    start_date: str,
    end_date: str
) -> Tuple[Dict[str, pd.DataFrame], List[str]]:
    """
    Screen all stocks in universe and return qualified candidates.
    
    Returns:
        (all_data_dict, qualified_symbols_list)
    """
    all_data = {}
    qualified = []
    
    print(f"Screening {len(universe)} stocks...\n")
    
    for symbol, name in universe.items():
        # Download and process
        data = download_stock_data(symbol, start_date, end_date)
        
        if data is None:
            print(f"✗ {symbol:12} ({name:25}) - Data unavailable")
            continue
        
        data = calculate_indicators(data)
        all_data[symbol] = data
        
        # Screen
        passed, criteria = screen_stock(data)
        
        if passed:
            qualified.append(symbol)
            status = "✓ QUALIFIED"
        else:
            status = "✗ Not qualified"
        
        print(f"{status:15} {symbol:12} ({name:25}) Price: RM {criteria['current_price']:.2f}, RSI: {criteria['current_rsi']:.1f}")
    
    return all_data, qualified


# Run universe screening
print("Running Complete Universe Screening")
print("=" * 80)

stock_data_dict, qualified_stocks = screen_universe(
    STOCK_UNIVERSE,
    START_DATE,
    END_DATE
)

print("\n" + "=" * 80)
print(f"Screening Complete: {len(qualified_stocks)} out of {len(STOCK_UNIVERSE)} stocks qualified")
print("\nQualified Stocks for Trading:")
for symbol in qualified_stocks:
    print(f"  • {symbol} - {STOCK_UNIVERSE[symbol]}")

---

## Part 2: Entry Signal Generation

Generate buy/sell signals using multiple confirmation indicators.

### Buy Signal Requirements (All must be true)

1. **Trend**: Price above MA50, MA20 > MA50
2. **Momentum**: RSI between 30-70 (not overbought/oversold)
3. **MACD**: Histogram positive (bullish momentum)
4. **Volume**: Above average (confirmation)
5. **Pattern**: Price pulled back to MA20 support (entry point)

### Sell Signal Requirements

1. **Exit**: Stop-loss or take-profit hit
2. **Trend reversal**: MA20 crosses below MA50
3. **Momentum loss**: MACD histogram turns negative
4. **Overbought**: RSI > 75

In [ ]:
def generate_signals(data: pd.DataFrame) -> pd.DataFrame:
    """
    Generate buy/sell signals based on multiple technical indicators.
    
    Adds 'signal' column: 1 = buy, -1 = sell, 0 = hold
    """
    df = data.copy()
    
    # Initialize signal column
    df['signal'] = 0
    
    # Buy conditions (all must be true)
    buy_conditions = (
        (df['close'] > df['ma_long']) &  # Uptrend
        (df['ma_short'] > df['ma_long']) &  # MA bullish alignment
        (df['rsi'] > PARAMS['rsi_oversold']) &  # Not oversold
        (df['rsi'] < PARAMS['rsi_overbought']) &  # Not overbought
        (df['macd_histogram'] > 0) &  # Bullish MACD
        (df['volume_ratio'] > 1.0) &  # Above average volume
        # Entry trigger: price near MA20 (within 2% for good entry)
        (np.abs(df['close'] - df['ma_short']) / df['ma_short'] < 0.02)
    )
    
    # Sell conditions (any can trigger)
    sell_conditions = (
        (df['ma_short'] < df['ma_long']) |  # Trend reversal
        (df['macd_histogram'] < 0) |  # Momentum loss
        (df['rsi'] > 75)  # Extremely overbought
    )
    
    # Assign signals
    df.loc[buy_conditions, 'signal'] = 1
    df.loc[sell_conditions, 'signal'] = -1
    
    return df


# Test signal generation on a qualified stock
if qualified_stocks:
    test_symbol = qualified_stocks[0]
    test_data = stock_data_dict[test_symbol]
    test_data = generate_signals(test_data)
    
    # Count signals
    buy_signals = (test_data['signal'] == 1).sum()
    sell_signals = (test_data['signal'] == -1).sum()
    
    print(f"Signal Generation Test - {test_symbol} ({STOCK_UNIVERSE[test_symbol]})")
    print("=" * 60)
    print(f"Period: {test_data.index[0].date()} to {test_data.index[-1].date()}")
    print(f"Total Trading Days: {len(test_data)}")
    print(f"Buy Signals: {buy_signals}")
    print(f"Sell Signals: {sell_signals}")
    print(f"Signal Frequency: {buy_signals / len(test_data) * 100:.2f}% of days")
    
    # Show recent signals
    recent_signals = test_data[test_data['signal'] != 0].tail(10)
    if not recent_signals.empty:
        print("\nRecent Signals:")
        print(recent_signals[['close', 'ma_short', 'rsi', 'macd_histogram', 'signal']])
else:
    print("No qualified stocks available for signal testing.")

---

## Part 3: Position Sizing Calculator

Calculate how much to invest in each trade using the **2% risk rule**.

### Position Sizing Formula

```
Position Size = (Account Value × Risk %) / (Entry Price - Stop Loss Price)
```

Where:
- **Account Value**: Current portfolio value
- **Risk %**: Maximum risk per trade (2%)
- **Stop Loss**: Entry Price - (2 × ATR)

### Example

- Account: RM 100,000
- Risk per trade: 2% = RM 2,000
- Entry price: RM 5.00
- ATR: RM 0.15
- Stop loss: RM 5.00 - (2 × RM 0.15) = RM 4.70
- Risk per share: RM 0.30
- Position size: RM 2,000 / RM 0.30 = 6,667 shares = RM 33,333 investment

In [ ]:
def calculate_position_size(
    account_value: float,
    entry_price: float,
    atr: float,
    risk_percent: float = MAX_RISK_PER_TRADE,
    atr_multiplier: float = STOP_LOSS_ATR_MULTIPLIER
) -> Dict[str, float]:
    """
    Calculate position size using ATR-based stop loss.
    
    Returns dictionary with position details.
    """
    # Calculate stop loss price
    stop_loss_distance = atr * atr_multiplier
    stop_loss_price = entry_price - stop_loss_distance
    
    # Risk amount in RM
    risk_amount = account_value * risk_percent
    
    # Risk per share
    risk_per_share = entry_price - stop_loss_price
    
    # Position size in shares
    shares = risk_amount / risk_per_share
    shares = int(shares)  # Round down to whole shares
    
    # Position value
    position_value = shares * entry_price
    
    # Calculate take profit (reward:risk = 1.5:1)
    take_profit_price = entry_price + (stop_loss_distance * 1.5)
    
    # Transaction costs
    entry_cost = position_value * TOTAL_COST_PER_TRADE
    exit_cost = position_value * TOTAL_COST_PER_TRADE
    total_costs = entry_cost + exit_cost
    
    return {
        'shares': shares,
        'position_value': position_value,
        'entry_price': entry_price,
        'stop_loss': stop_loss_price,
        'take_profit': take_profit_price,
        'risk_amount': shares * risk_per_share,
        'reward_potential': shares * (take_profit_price - entry_price),
        'transaction_costs': total_costs,
        'risk_reward_ratio': 1.5,
        'position_pct': position_value / account_value * 100,
    }


# Test position sizing
print("Position Sizing Calculator Test")
print("=" * 60)

# Example calculation
example_account = 100000
example_entry = 5.00
example_atr = 0.15

position = calculate_position_size(
    account_value=example_account,
    entry_price=example_entry,
    atr=example_atr
)

print(f"Account Value: RM {example_account:,.0f}")
print(f"Entry Price: RM {example_entry:.2f}")
print(f"ATR: RM {example_atr:.2f}")
print("\nPosition Sizing Results:")
print(f"  Shares to Buy: {position['shares']:,}")
print(f"  Position Value: RM {position['position_value']:,.0f} ({position['position_pct']:.1f}% of account)")
print(f"  Entry Price: RM {position['entry_price']:.2f}")
print(f"  Stop Loss: RM {position['stop_loss']:.2f} ({(position['stop_loss']/position['entry_price']-1)*100:.1f}%)")
print(f"  Take Profit: RM {position['take_profit']:.2f} (+{(position['take_profit']/position['entry_price']-1)*100:.1f}%)")
print(f"  Risk Amount: RM {position['risk_amount']:,.0f}")
print(f"  Reward Potential: RM {position['reward_potential']:,.0f}")
print(f"  Risk:Reward Ratio: 1:{position['risk_reward_ratio']:.1f}")
print(f"  Transaction Costs: RM {position['transaction_costs']:.2f}")

---

## Part 4: Trade Management System

Build a class to manage individual trades with dynamic stops and targets.

In [ ]:
class Trade:
    """
    Represents a single trade with entry, exit, and management logic.
    """
    
    def __init__(self, symbol: str, entry_date, position_details: Dict):
        self.symbol = symbol
        self.entry_date = entry_date
        self.exit_date = None
        
        # Position details
        self.shares = position_details['shares']
        self.entry_price = position_details['entry_price']
        self.stop_loss = position_details['stop_loss']
        self.take_profit = position_details['take_profit']
        self.initial_stop = position_details['stop_loss']
        
        # Costs and P&L
        self.entry_cost = position_details['position_value'] * TOTAL_COST_PER_TRADE
        self.exit_cost = 0
        self.exit_price = None
        self.pnl = 0
        self.pnl_percent = 0
        
        # Status
        self.is_open = True
        self.exit_reason = None
        self.highest_price = entry_price  # For trailing stop
        
    def update(self, current_date, high: float, low: float, close: float):
        """
        Update trade status with current price data.
        Returns True if trade should be closed.
        """
        if not self.is_open:
            return False
        
        # Update highest price for trailing stop
        if high > self.highest_price:
            self.highest_price = high
            
            # Activate trailing stop if profit > 1.5x ATR
            profit_distance = self.highest_price - self.entry_price
            initial_stop_distance = self.entry_price - self.initial_stop
            
            if profit_distance > (initial_stop_distance * TRAILING_STOP_ATR_MULTIPLIER):
                # Trail stop loss up (never down)
                new_stop = self.highest_price - initial_stop_distance
                if new_stop > self.stop_loss:
                    self.stop_loss = new_stop
        
        # Check exit conditions
        if low <= self.stop_loss:
            # Stop loss hit
            self.close_trade(current_date, self.stop_loss, 'stop_loss')
            return True
            
        elif high >= self.take_profit:
            # Take profit hit
            self.close_trade(current_date, self.take_profit, 'take_profit')
            return True
        
        return False
    
    def close_trade(self, exit_date, exit_price: float, reason: str):
        """
        Close the trade and calculate P&L.
        """
        self.is_open = False
        self.exit_date = exit_date
        self.exit_price = exit_price
        self.exit_reason = reason
        
        # Calculate exit cost
        exit_value = self.shares * exit_price
        self.exit_cost = exit_value * TOTAL_COST_PER_TRADE
        
        # Calculate P&L
        gross_pnl = (exit_price - self.entry_price) * self.shares
        total_costs = self.entry_cost + self.exit_cost
        self.pnl = gross_pnl - total_costs
        
        # Calculate percentage return
        entry_value = self.shares * self.entry_price
        self.pnl_percent = (self.pnl / entry_value) * 100
    
    def get_summary(self) -> Dict:
        """
        Get trade summary as dictionary.
        """
        return {
            'symbol': self.symbol,
            'entry_date': self.entry_date,
            'exit_date': self.exit_date,
            'shares': self.shares,
            'entry_price': self.entry_price,
            'exit_price': self.exit_price,
            'exit_reason': self.exit_reason,
            'pnl': self.pnl,
            'pnl_percent': self.pnl_percent,
            'days_held': (self.exit_date - self.entry_date).days if self.exit_date else None,
        }


# Test trade management
print("Trade Management System Test")
print("=" * 60)

# Create example trade
example_position = calculate_position_size(
    account_value=100000,
    entry_price=5.00,
    atr=0.15
)

test_trade = Trade(
    symbol='1155.KL',
    entry_date=datetime(2023, 1, 1),
    position_details=example_position
)

print(f"Trade Created: {test_trade.symbol}")
print(f"Entry: RM {test_trade.entry_price:.2f} × {test_trade.shares:,} shares")
print(f"Stop Loss: RM {test_trade.stop_loss:.2f}")
print(f"Take Profit: RM {test_trade.take_profit:.2f}")
print(f"Status: {'OPEN' if test_trade.is_open else 'CLOSED'}")

# Simulate price movement hitting take profit
test_trade.update(
    current_date=datetime(2023, 1, 15),
    high=5.50,  # Hits take profit
    low=5.20,
    close=5.45
)

print("\nAfter Price Update (Take Profit Hit):")
summary = test_trade.get_summary()
print(f"Exit: RM {summary['exit_price']:.2f} on {summary['exit_date'].date()}")
print(f"Reason: {summary['exit_reason']}")
print(f"P&L: RM {summary['pnl']:,.2f} ({summary['pnl_percent']:+.2f}%)")
print(f"Days Held: {summary['days_held']}")

---

## Part 5: Multi-Stock Portfolio Backtesting

Backtest the complete trading system across multiple stocks simultaneously.

### Portfolio Rules

1. **Maximum positions**: 5 concurrent trades
2. **Capital allocation**: Dynamic based on account value
3. **Risk management**: Maximum 6% total portfolio risk
4. **Trade selection**: Priority to stocks with strongest signals

In [ ]:
class TradingSystem:
    """
    Complete trading system managing multiple stocks and positions.
    """
    
    def __init__(self, initial_capital: float, max_positions: int = MAX_POSITIONS):
        self.initial_capital = initial_capital
        self.cash = initial_capital
        self.max_positions = max_positions
        
        # Trading tracking
        self.open_trades: List[Trade] = []
        self.closed_trades: List[Trade] = []
        
        # Performance tracking
        self.equity_curve = []
        self.dates = []
        
    def get_portfolio_value(self, current_prices: Dict[str, float]) -> float:
        """
        Calculate total portfolio value (cash + open positions).
        """
        open_value = sum(
            trade.shares * current_prices.get(trade.symbol, trade.entry_price)
            for trade in self.open_trades
        )
        return self.cash + open_value
    
    def can_open_position(self) -> bool:
        """
        Check if we can open a new position.
        """
        return len(self.open_trades) < self.max_positions
    
    def open_position(self, symbol: str, entry_date, current_price: float, atr: float):
        """
        Open a new trading position.
        """
        if not self.can_open_position():
            return False
        
        # Calculate position size based on current account value
        account_value = self.cash + sum(
            trade.shares * current_price for trade in self.open_trades
        )
        
        position_details = calculate_position_size(
            account_value=account_value,
            entry_price=current_price,
            atr=atr
        )
        
        # Check if we have enough cash
        required_cash = position_details['position_value'] + position_details['transaction_costs']
        if required_cash > self.cash:
            return False
        
        # Create and open trade
        trade = Trade(symbol, entry_date, position_details)
        self.open_trades.append(trade)
        self.cash -= required_cash
        
        return True
    
    def update_positions(self, current_date, price_data: Dict[str, Dict]):
        """
        Update all open positions with current price data.
        """
        for trade in self.open_trades[:]:
            if trade.symbol in price_data:
                data = price_data[trade.symbol]
                should_close = trade.update(
                    current_date,
                    high=data['high'],
                    low=data['low'],
                    close=data['close']
                )
                
                if should_close:
                    # Move to closed trades and free up cash
                    self.open_trades.remove(trade)
                    self.closed_trades.append(trade)
                    self.cash += (trade.shares * trade.exit_price) - trade.exit_cost
    
    def record_equity(self, date, current_prices: Dict[str, float]):
        """
        Record current portfolio equity for tracking.
        """
        equity = self.get_portfolio_value(current_prices)
        self.dates.append(date)
        self.equity_curve.append(equity)


def backtest_strategy(
    stock_data: Dict[str, pd.DataFrame],
    initial_capital: float = INITIAL_CAPITAL
) -> TradingSystem:
    """
    Run backtest across all stocks in portfolio.
    """
    # Initialize trading system
    system = TradingSystem(initial_capital)
    
    # Generate signals for all stocks
    for symbol in stock_data:
        stock_data[symbol] = generate_signals(stock_data[symbol])
    
    # Get common date range
    all_dates = sorted(set(
        date for df in stock_data.values() for date in df.index
    ))
    
    print(f"Backtesting {len(stock_data)} stocks over {len(all_dates)} trading days...")
    
    # Iterate through each trading day
    for current_date in all_dates:
        # Collect current price data
        current_prices = {}
        price_data = {}
        
        for symbol, df in stock_data.items():
            if current_date in df.index:
                row = df.loc[current_date]
                current_prices[symbol] = row['close']
                price_data[symbol] = {
                    'high': row['high'],
                    'low': row['low'],
                    'close': row['close'],
                }
        
        # Update existing positions
        system.update_positions(current_date, price_data)
        
        # Check for new entry signals
        for symbol, df in stock_data.items():
            if current_date not in df.index:
                continue
            
            row = df.loc[current_date]
            
            # Buy signal and can open position
            if row['signal'] == 1 and system.can_open_position():
                # Check not already in this stock
                if not any(t.symbol == symbol for t in system.open_trades):
                    system.open_position(
                        symbol=symbol,
                        entry_date=current_date,
                        current_price=row['close'],
                        atr=row['atr']
                    )
        
        # Record portfolio equity
        system.record_equity(current_date, current_prices)
    
    return system


# Run backtest on available data
print("Running Multi-Stock Portfolio Backtest")
print("=" * 80)

if stock_data_dict:
    trading_system = backtest_strategy(stock_data_dict, INITIAL_CAPITAL)
    
    print("\nBacktest Complete!")
    print("=" * 80)
    print(f"Initial Capital: RM {trading_system.initial_capital:,.0f}")
    print(f"Final Equity: RM {trading_system.equity_curve[-1]:,.0f}")
    print(f"Total Return: {(trading_system.equity_curve[-1] / trading_system.initial_capital - 1) * 100:+.2f}%")
    print(f"\nTotal Trades: {len(trading_system.closed_trades)}")
    print(f"Open Positions: {len(trading_system.open_trades)}")
    
    if trading_system.closed_trades:
        winning_trades = [t for t in trading_system.closed_trades if t.pnl > 0]
        print(f"Winning Trades: {len(winning_trades)} ({len(winning_trades)/len(trading_system.closed_trades)*100:.1f}%)")
else:
    print("No stock data available for backtesting.")

---

## Part 6: Performance Analysis

Comprehensive analysis of strategy performance with all key metrics.

In [ ]:
def calculate_performance_metrics(system: TradingSystem) -> Dict:
    """
    Calculate comprehensive performance metrics.
    """
    if not system.equity_curve or not system.closed_trades:
        return {}
    
    # Convert equity curve to returns
    equity_series = pd.Series(system.equity_curve, index=system.dates)
    returns = equity_series.pct_change().dropna()
    
    # Total return
    total_return = (equity_series.iloc[-1] / equity_series.iloc[0] - 1) * 100
    
    # Annualized return (assuming ~252 trading days per year)
    days = (system.dates[-1] - system.dates[0]).days
    years = days / 365.25
    annualized_return = ((equity_series.iloc[-1] / equity_series.iloc[0]) ** (1/years) - 1) * 100
    
    # Volatility (annualized)
    daily_volatility = returns.std()
    annualized_volatility = daily_volatility * np.sqrt(252) * 100
    
    # Sharpe Ratio (assuming 3% risk-free rate for Malaysia)
    risk_free_rate = 0.03
    excess_return = annualized_return/100 - risk_free_rate
    sharpe_ratio = excess_return / (annualized_volatility/100) if annualized_volatility > 0 else 0
    
    # Maximum Drawdown
    cumulative_max = equity_series.expanding().max()
    drawdown = (equity_series - cumulative_max) / cumulative_max * 100
    max_drawdown = drawdown.min()
    
    # Win rate and trade statistics
    total_trades = len(system.closed_trades)
    winning_trades = [t for t in system.closed_trades if t.pnl > 0]
    losing_trades = [t for t in system.closed_trades if t.pnl <= 0]
    
    win_rate = len(winning_trades) / total_trades * 100 if total_trades > 0 else 0
    
    avg_win = np.mean([t.pnl for t in winning_trades]) if winning_trades else 0
    avg_loss = np.mean([t.pnl for t in losing_trades]) if losing_trades else 0
    
    profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else 0
    
    # Average holding period
    avg_holding_days = np.mean([t.get_summary()['days_held'] for t in system.closed_trades])
    
    return {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'total_trades': total_trades,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'profit_factor': profit_factor,
        'avg_holding_days': avg_holding_days,
    }


def plot_performance(system: TradingSystem, benchmark_data: pd.DataFrame = None):
    """
    Create comprehensive performance visualization.
    """
    fig = plt.figure(figsize=(16, 12))
    gs = gridspec.GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.3)
    
    # 1. Equity Curve
    ax1 = fig.add_subplot(gs[0, :])
    equity_series = pd.Series(system.equity_curve, index=system.dates)
    ax1.plot(equity_series.index, equity_series.values, label='Strategy', linewidth=2)
    
    # Add benchmark if available
    if benchmark_data is not None:
        # Normalize benchmark to same starting value
        benchmark_aligned = benchmark_data.reindex(equity_series.index, method='ffill')
        benchmark_normalized = (
            benchmark_aligned['close'] / benchmark_aligned['close'].iloc[0] * system.initial_capital
        )
        ax1.plot(benchmark_normalized.index, benchmark_normalized.values, 
                label='FBM KLCI', linewidth=2, alpha=0.7, linestyle='--')
    
    ax1.axhline(y=system.initial_capital, color='gray', linestyle=':', alpha=0.5, label='Initial Capital')
    ax1.set_title('Portfolio Equity Curve', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Portfolio Value (RM)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'RM {x/1000:.0f}K'))
    
    # 2. Drawdown
    ax2 = fig.add_subplot(gs[1, 0])
    cumulative_max = equity_series.expanding().max()
    drawdown = (equity_series - cumulative_max) / cumulative_max * 100
    ax2.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
    ax2.plot(drawdown.index, drawdown.values, color='red', linewidth=1)
    ax2.set_title('Drawdown', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Drawdown (%)')
    ax2.grid(True, alpha=0.3)
    
    # 3. Monthly Returns Heatmap
    ax3 = fig.add_subplot(gs[1, 1])
    monthly_returns = equity_series.resample('M').last().pct_change() * 100
    monthly_returns_pivot = monthly_returns.to_frame('returns')
    monthly_returns_pivot['year'] = monthly_returns_pivot.index.year
    monthly_returns_pivot['month'] = monthly_returns_pivot.index.month
    
    if len(monthly_returns_pivot) > 1:
        pivot_table = monthly_returns_pivot.pivot_table(
            values='returns', 
            index='year', 
            columns='month'
        )
        sns.heatmap(pivot_table, annot=True, fmt='.1f', cmap='RdYlGn', center=0, 
                   ax=ax3, cbar_kws={'label': 'Return (%)'})
        ax3.set_title('Monthly Returns Heatmap', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Month')
        ax3.set_ylabel('Year')
    
    # 4. Trade Distribution
    ax4 = fig.add_subplot(gs[2, 0])
    trade_returns = [t.pnl_percent for t in system.closed_trades]
    if trade_returns:
        ax4.hist(trade_returns, bins=20, alpha=0.7, edgecolor='black')
        ax4.axvline(x=0, color='red', linestyle='--', linewidth=2)
        ax4.set_title('Distribution of Trade Returns', fontsize=12, fontweight='bold')
        ax4.set_xlabel('Return (%)')
        ax4.set_ylabel('Frequency')
        ax4.grid(True, alpha=0.3)
    
    # 5. Performance Metrics Table
    ax5 = fig.add_subplot(gs[2, 1])
    ax5.axis('off')
    
    metrics = calculate_performance_metrics(system)
    if metrics:
        table_data = [
            ['Total Return', f"{metrics['total_return']:+.2f}%"],
            ['Annualized Return', f"{metrics['annualized_return']:+.2f}%"],
            ['Volatility (Annual)', f"{metrics['annualized_volatility']:.2f}%"],
            ['Sharpe Ratio', f"{metrics['sharpe_ratio']:.2f}"],
            ['Max Drawdown', f"{metrics['max_drawdown']:.2f}%"],
            ['Total Trades', f"{metrics['total_trades']}"],
            ['Win Rate', f"{metrics['win_rate']:.1f}%"],
            ['Avg Win', f"RM {metrics['avg_win']:.0f}"],
            ['Avg Loss', f"RM {metrics['avg_loss']:.0f}"],
            ['Profit Factor', f"{metrics['profit_factor']:.2f}"],
            ['Avg Hold Days', f"{metrics['avg_holding_days']:.1f}"],
        ]
        
        table = ax5.table(cellText=table_data, cellLoc='left',
                         colWidths=[0.6, 0.4], loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        
        # Color code rows
        for i in range(len(table_data)):
            table[(i, 0)].set_facecolor('#E8E8E8')
            table[(i, 1)].set_facecolor('#F5F5F5')
    
    plt.suptitle('Trading System Performance Dashboard', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()


# Analyze performance
if 'trading_system' in locals() and trading_system.closed_trades:
    print("\nPerformance Analysis")
    print("=" * 80)
    
    metrics = calculate_performance_metrics(trading_system)
    
    print("\nReturn Metrics:")
    print(f"  Total Return: {metrics['total_return']:+.2f}%")
    print(f"  Annualized Return: {metrics['annualized_return']:+.2f}%")
    print(f"  Annualized Volatility: {metrics['annualized_volatility']:.2f}%")
    print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"  Maximum Drawdown: {metrics['max_drawdown']:.2f}%")
    
    print("\nTrade Metrics:")
    print(f"  Total Trades: {metrics['total_trades']}")
    print(f"  Win Rate: {metrics['win_rate']:.1f}%")
    print(f"  Average Win: RM {metrics['avg_win']:,.2f}")
    print(f"  Average Loss: RM {metrics['avg_loss']:,.2f}")
    print(f"  Profit Factor: {metrics['profit_factor']:.2f}")
    print(f"  Average Holding Period: {metrics['avg_holding_days']:.1f} days")
    
    # Download benchmark for comparison
    benchmark = download_stock_data(BENCHMARK_SYMBOL, START_DATE, END_DATE)
    
    # Plot comprehensive dashboard
    plot_performance(trading_system, benchmark)

### Trade-by-Trade Analysis

Detailed breakdown of all closed trades.

In [ ]:
# Create detailed trade log
if 'trading_system' in locals() and trading_system.closed_trades:
    trade_log = pd.DataFrame([t.get_summary() for t in trading_system.closed_trades])
    
    # Add stock names
    trade_log['stock_name'] = trade_log['symbol'].map(STOCK_UNIVERSE)
    
    # Reorder columns
    columns_order = [
        'symbol', 'stock_name', 'entry_date', 'exit_date', 'days_held',
        'shares', 'entry_price', 'exit_price', 'exit_reason', 'pnl', 'pnl_percent'
    ]
    trade_log = trade_log[columns_order]
    
    print("\nTrade-by-Trade Log")
    print("=" * 120)
    print(trade_log.to_string(index=False))
    
    # Summary by stock
    print("\n\nPerformance by Stock")
    print("=" * 80)
    
    stock_summary = trade_log.groupby(['symbol', 'stock_name']).agg({
        'pnl': ['count', 'sum', 'mean'],
        'pnl_percent': 'mean',
        'days_held': 'mean'
    }).round(2)
    
    stock_summary.columns = ['Trades', 'Total P&L (RM)', 'Avg P&L (RM)', 'Avg Return (%)', 'Avg Days']
    print(stock_summary)
    
    # Summary by exit reason
    print("\n\nPerformance by Exit Reason")
    print("=" * 80)
    
    exit_summary = trade_log.groupby('exit_reason').agg({
        'pnl': ['count', 'sum', 'mean'],
        'pnl_percent': 'mean',
        'days_held': 'mean'
    }).round(2)
    
    exit_summary.columns = ['Count', 'Total P&L (RM)', 'Avg P&L (RM)', 'Avg Return (%)', 'Avg Days']
    print(exit_summary)

---

## Part 7: Risk Management Dashboard

Real-time risk monitoring for portfolio management.

In [ ]:
def create_risk_dashboard(system: TradingSystem, current_prices: Dict[str, float]):
    """
    Create comprehensive risk management dashboard.
    """
    portfolio_value = system.get_portfolio_value(current_prices)
    
    print("RISK MANAGEMENT DASHBOARD")
    print("=" * 80)
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Portfolio Overview
    print("Portfolio Overview")
    print("-" * 80)
    print(f"Total Portfolio Value: RM {portfolio_value:,.2f}")
    print(f"Cash Available: RM {system.cash:,.2f} ({system.cash/portfolio_value*100:.1f}%)")
    print(f"Open Positions: {len(system.open_trades)} / {system.max_positions} max")
    print(f"Closed Trades: {len(system.closed_trades)}")
    print()
    
    # Current Positions
    if system.open_trades:
        print("Current Open Positions")
        print("-" * 80)
        
        position_data = []
        total_risk = 0
        
        for trade in system.open_trades:
            current_price = current_prices.get(trade.symbol, trade.entry_price)
            position_value = trade.shares * current_price
            unrealized_pnl = (current_price - trade.entry_price) * trade.shares
            unrealized_pnl_pct = (current_price / trade.entry_price - 1) * 100
            
            # Calculate current risk
            current_risk = (current_price - trade.stop_loss) * trade.shares
            risk_pct = current_risk / portfolio_value * 100
            total_risk += risk_pct
            
            position_data.append({
                'Symbol': trade.symbol,
                'Stock': STOCK_UNIVERSE[trade.symbol],
                'Shares': trade.shares,
                'Entry': f"RM {trade.entry_price:.2f}",
                'Current': f"RM {current_price:.2f}",
                'Stop Loss': f"RM {trade.stop_loss:.2f}",
                'Position Value': f"RM {position_value:,.0f}",
                'Unrealized P&L': f"RM {unrealized_pnl:+,.0f} ({unrealized_pnl_pct:+.1f}%)",
                'Risk': f"{risk_pct:.2f}%",
            })
        
        positions_df = pd.DataFrame(position_data)
        print(positions_df.to_string(index=False))
        print()
        print(f"Total Portfolio Risk: {total_risk:.2f}% (Max allowed: {MAX_PORTFOLIO_RISK*100}%)")
        
        if total_risk > MAX_PORTFOLIO_RISK * 100:
            print("⚠️  WARNING: Portfolio risk exceeds maximum threshold!")
        print()
    else:
        print("No open positions.\n")
    
    # Performance Summary
    if system.closed_trades:
        print("Recent Performance (Last 10 Trades)")
        print("-" * 80)
        
        recent_trades = system.closed_trades[-10:]
        recent_data = []
        
        for trade in recent_trades:
            recent_data.append({
                'Date': trade.exit_date.strftime('%Y-%m-%d'),
                'Symbol': trade.symbol,
                'Days': (trade.exit_date - trade.entry_date).days,
                'Exit': trade.exit_reason,
                'P&L': f"RM {trade.pnl:+,.0f}",
                'Return': f"{trade.pnl_percent:+.2f}%",
            })
        
        recent_df = pd.DataFrame(recent_data)
        print(recent_df.to_string(index=False))
        print()
    
    # Risk Alerts
    print("Risk Alerts")
    print("-" * 80)
    
    alerts = []
    
    # Check drawdown
    if system.equity_curve:
        peak_equity = max(system.equity_curve)
        current_drawdown = (portfolio_value - peak_equity) / peak_equity * 100
        
        if current_drawdown < -10:
            alerts.append(f"⚠️  HIGH DRAWDOWN: {current_drawdown:.1f}% from peak")
    
    # Check concentration
    if system.open_trades:
        max_position = max(
            trade.shares * current_prices.get(trade.symbol, trade.entry_price)
            for trade in system.open_trades
        )
        concentration = max_position / portfolio_value * 100
        
        if concentration > 30:
            alerts.append(f"⚠️  HIGH CONCENTRATION: {concentration:.1f}% in single position")
    
    # Check cash level
    cash_pct = system.cash / portfolio_value * 100
    if cash_pct < 10 and len(system.open_trades) > 0:
        alerts.append(f"⚠️  LOW CASH: Only {cash_pct:.1f}% available")
    
    if alerts:
        for alert in alerts:
            print(alert)
    else:
        print("✓ No risk alerts. Portfolio within acceptable parameters.")
    
    print("\n" + "=" * 80)


# Display risk dashboard
if 'trading_system' in locals():
    # Get latest prices
    latest_prices = {
        symbol: df['close'].iloc[-1] 
        for symbol, df in stock_data_dict.items()
    }
    
    create_risk_dashboard(trading_system, latest_prices)

---

## Part 8: Paper Trading Recommendations

Before trading with real money, practice with paper trading.

### Paper Trading Platforms in Malaysia

1. **Bursa Anywhere** (Official Bursa Malaysia)
   - Virtual trading platform
   - Real-time market data
   - Free to use
   - Website: https://www.bursamarketplace.com/

2. **Rakuten Trade** 
   - Demo account available
   - Low brokerage fees (0.05%)
   - Good for practice

3. **M+ Online**
   - Maybank investment platform
   - Paper trading mode
   - Research tools included

### Paper Trading Checklist

Use this system for paper trading:

1. ✅ **Screen stocks** using the criteria from Part 1
2. ✅ **Wait for signals** - don't force trades
3. ✅ **Calculate position size** before entering
4. ✅ **Set stop-loss** immediately after entry
5. ✅ **Set take-profit** target
6. ✅ **Track each trade** in a journal
7. ✅ **Review weekly** - what worked, what didn't
8. ✅ **Monitor risk** - stay within limits

### Trade Journal Template

Record every trade:

```
Trade #: ___
Date: ___________
Stock: ___________ (Symbol: _______)
Entry Price: RM _______
Position Size: _______ shares (RM _______)
Stop Loss: RM _______ (-_____%)  
Take Profit: RM _______ (+_____%)  
Risk Amount: RM _______ (_____% of account)

Entry Reason (Indicators):
- MA: ___________
- RSI: ___________  
- MACD: ___________
- Volume: ___________
- Pattern: ___________

Exit Date: ___________
Exit Price: RM _______
Exit Reason: ___________
P&L: RM _______ (_____%)  

What went well:
_______________________________

What to improve:
_______________________________
```

### When to Transition to Real Money

**Only start live trading when you can answer YES to all:**

- [ ] Completed 30+ paper trades
- [ ] Achieved positive returns over 3+ months
- [ ] Win rate > 40%
- [ ] Followed risk rules consistently (no violations)
- [ ] Understand why each trade won or lost
- [ ] Comfortable with maximum drawdown
- [ ] Can execute system without emotional decisions
- [ ] Have emergency fund (6 months expenses)
- [ ] Using only risk capital (can afford to lose)

**Start small**: Begin with 25-50% of intended capital.

In [ ]:
# Generate current trading signals for paper trading
def generate_current_opportunities():
    """
    Scan for current trading opportunities (for paper trading).
    """
    print("Current Trading Opportunities (Paper Trading)")
    print("=" * 80)
    print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d')}")
    print()
    
    opportunities = []
    
    for symbol, df in stock_data_dict.items():
        # Get latest data
        latest = df.iloc[-1]
        
        # Check for buy signal
        if latest['signal'] == 1:
            # Calculate suggested position
            position = calculate_position_size(
                account_value=INITIAL_CAPITAL,
                entry_price=latest['close'],
                atr=latest['atr']
            )
            
            opportunities.append({
                'Symbol': symbol,
                'Stock': STOCK_UNIVERSE[symbol],
                'Current Price': f"RM {latest['close']:.2f}",
                'RSI': f"{latest['rsi']:.1f}",
                'MACD': 'Bullish' if latest['macd_histogram'] > 0 else 'Bearish',
                'Suggested Shares': position['shares'],
                'Position Value': f"RM {position['position_value']:,.0f}",
                'Stop Loss': f"RM {position['stop_loss']:.2f}",
                'Take Profit': f"RM {position['take_profit']:.2f}",
            })
    
    if opportunities:
        opp_df = pd.DataFrame(opportunities)
        print(f"Found {len(opportunities)} potential opportunities:\n")
        print(opp_df.to_string(index=False))
        print("\n⚠️  These are for PAPER TRADING practice only!")
        print("⚠️  Always verify signals before any real trades.")
    else:
        print("No buy signals in current market conditions.")
        print("Wait for clear setups - patience is key!")
    
    print("\n" + "=" * 80)


# Generate opportunities
if stock_data_dict:
    generate_current_opportunities()

---

## Advanced Challenge Exercises

Extend the trading system with these advanced features.

### Exercise 1: Add Sector Rotation

**Objective**: Enhance screening to favor sectors showing strength.

**Tasks**:
1. Group stocks by sector (banking, plantation, construction, etc.)
2. Calculate sector momentum (average sector return over 20 days)
3. Increase position sizing for stocks in strong sectors by 20%
4. Reduce position sizing for stocks in weak sectors by 20%

**Hint**: Create sector mapping dictionary and calculate relative strength.

```python
SECTORS = {
    'Banking': ['1155.KL', '1023.KL', '1295.KL'],
    'Plantation': ['5285.KL', '1961.KL'],
    # Add others...
}

def calculate_sector_strength(sector_stocks, data_dict):
    # Your code here
    pass
```

In [ ]:
# Exercise 1: Your solution here


### Exercise 2: Implement Portfolio Rebalancing

**Objective**: Automatically rebalance portfolio when positions become uneven.

**Tasks**:
1. Calculate each position's percentage of total portfolio
2. If any position exceeds 30% due to price appreciation, reduce it
3. If any position falls below 10%, consider closing
4. Redistribute capital to maintain balanced exposure

**Hint**: Modify the `TradingSystem` class to add `rebalance_portfolio()` method.

```python
def rebalance_portfolio(self, current_prices, max_position_pct=0.30):
    # Your code here
    pass
```

In [ ]:
# Exercise 2: Your solution here


### Exercise 3: Monte Carlo Simulation

**Objective**: Assess strategy robustness using Monte Carlo analysis.

**Tasks**:
1. Run backtest 1000 times with randomized trade order
2. Calculate distribution of final returns
3. Determine 5th percentile (worst case) and 95th percentile (best case)
4. Visualize distribution of outcomes

**Hint**: Shuffle trade entry dates while maintaining same trades.

```python
def monte_carlo_analysis(system, num_simulations=1000):
    results = []
    for i in range(num_simulations):
        # Randomize and backtest
        # Your code here
        pass
    return results
```

In [ ]:
# Exercise 3: Your solution here


### Exercise 4: Correlation-Based Position Limits

**Objective**: Avoid over-concentration in correlated stocks.

**Tasks**:
1. Calculate correlation matrix for all stocks
2. Before opening position, check correlation with existing positions
3. If correlation > 0.7 with any existing position, reduce size by 50%
4. If correlation > 0.85, skip the trade

**Hint**: Use pandas `.corr()` method on returns.

```python
def calculate_correlations(stock_data):
    # Calculate return correlations
    # Your code here
    pass

def check_correlation_risk(new_stock, existing_positions, correlation_matrix):
    # Your code here
    pass
```

In [ ]:
# Exercise 4: Your solution here


### Exercise 5: Machine Learning Enhancement

**Objective**: Use ML to improve signal quality.

**Tasks**:
1. Create features: RSI, MACD, volume ratio, ATR, MA slopes
2. Label historical signals as winners (return > 3%) or losers
3. Train Random Forest classifier to predict signal quality
4. Only take trades with ML confidence > 60%
5. Compare ML-enhanced vs original strategy

**Hint**: Use scikit-learn's `RandomForestClassifier`.

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def create_ml_features(data):
    # Your code here
    pass

def train_signal_classifier(features, labels):
    # Your code here
    pass
```

In [ ]:
# Exercise 5: Your solution here


---

## Summary and Graduation

### What You've Accomplished

**Congratulations! You've built a complete, professional-grade trading system!** 🎓🎉

In this module, you:

1. ✅ **Designed a stock screening system** that scans FBM KLCI constituents automatically
2. ✅ **Created multi-indicator entry signals** combining MA, RSI, MACD, volume, and patterns
3. ✅ **Implemented position sizing** using the 2% risk rule with ATR-based stops
4. ✅ **Built trade management** with dynamic stops, targets, and trailing stops
5. ✅ **Backtested across multiple stocks** with realistic Malaysian transaction costs
6. ✅ **Analyzed performance comprehensively** (returns, Sharpe, drawdown, win rate)
7. ✅ **Compared to FBM KLCI benchmark** to assess alpha generation
8. ✅ **Created risk dashboards** for real-time portfolio monitoring
9. ✅ **Prepared for paper trading** with clear checklists and guidelines
10. ✅ **Extended with advanced features** (optional exercises)

### Complete Trading Checklist

Use this checklist for every trading session:

**Before Market Open:**
- [ ] Review overnight news for portfolio stocks
- [ ] Check global markets (US, China, regional)
- [ ] Run stock screening for new opportunities
- [ ] Review open positions and adjust stops if needed
- [ ] Check available capital and position limits

**During Market Hours:**
- [ ] Monitor stop-loss levels on open positions
- [ ] Watch for take-profit targets being hit
- [ ] Execute new entries only on confirmed signals
- [ ] Avoid revenge trading after losses
- [ ] Stick to position sizing rules (no exceptions!)

**After Market Close:**
- [ ] Update trade journal for all activity
- [ ] Calculate daily P&L and drawdown
- [ ] Review what worked and what didn't
- [ ] Prepare watchlist for next session
- [ ] Update risk dashboard

**Weekly Review:**
- [ ] Calculate weekly performance metrics
- [ ] Review all closed trades
- [ ] Identify patterns in winners/losers
- [ ] Adjust strategy if needed (carefully!)
- [ ] Rebalance portfolio if required

### Key Principles to Remember

1. **Risk Management is Paramount**
   - Never risk more than 2% per trade
   - Never exceed 6% total portfolio risk
   - Cut losses quickly, let winners run

2. **Discipline Over Discretion**
   - Follow your system consistently
   - Don't override signals with "gut feel"
   - Trust the process, not individual trades

3. **Patience is Profitable**
   - Wait for quality setups
   - Don't force trades
   - Cash is a position

4. **Continuous Improvement**
   - Keep detailed records
   - Learn from every trade
   - Adapt to changing markets

5. **Psychological Strength**
   - Accept losses as cost of doing business
   - Don't let emotions drive decisions
   - Take breaks after big wins or losses

### Your Learning Journey

You started knowing nothing about technical analysis. Now you have:

- **Knowledge**: Understanding of indicators, patterns, risk management
- **Skills**: Ability to code, backtest, and analyze trading systems
- **Tools**: Complete working system ready for paper trading
- **Experience**: Hands-on practice with Malaysian stocks

### Next Steps

1. **Paper Trade for 3+ months**
   - Use Bursa Anywhere or broker demo account
   - Follow the system exactly
   - Keep detailed journal

2. **Continue Learning**
   - Read: "Technical Analysis of the Financial Markets" by John Murphy
   - Read: "Trading in the Zone" by Mark Douglas (psychology)
   - Follow: Malaysian investment blogs and forums

3. **Refine Your System**
   - Try different parameter combinations
   - Add indicators that resonate with you
   - Always backtest changes!

4. **Build Community**
   - Join Malaysian trading communities
   - Share knowledge (not tips!)
   - Learn from experienced traders

5. **Start Small with Real Money**
   - Only when paper trading is consistently profitable
   - Begin with 25-50% of intended capital
   - Scale up gradually as confidence grows

### Final Words

Trading is a journey, not a destination. You now have the tools, knowledge, and system to succeed. But remember:

**"The market doesn't care about your opinion, your hopes, or your fears. It only cares about supply and demand, buying and selling. Your job is to read these signals and respond accordingly."**

Stay disciplined. Manage risk. Keep learning. Be patient.

**You're ready. Good luck, and happy trading!** 🚀

---

*Disclaimer: This educational material is for learning purposes only. Trading involves substantial risk of loss. Never invest money you cannot afford to lose. Always do your own research and consider consulting a licensed financial advisor.*

---

## Additional Resources

### Malaysian Market Resources

**Official**:
- Bursa Malaysia: https://www.bursamalaysia.com/
- Securities Commission: https://www.sc.com.my/
- Bank Negara Malaysia: https://www.bnm.gov.my/

**Research & Data**:
- The Edge Markets: https://www.theedgemarkets.com/
- Investing.com Malaysia: https://www.investing.com/equities/malaysia
- Yahoo Finance: https://finance.yahoo.com/

**Learning**:
- Investopedia: https://www.investopedia.com/
- BabyPips (Forex, but good TA): https://www.babypips.com/learn
- TradingView Education: https://www.tradingview.com/education/

### Recommended Brokers in Malaysia

**Low-Cost Online Brokers**:
- Rakuten Trade (0.05% brokerage)
- M+ Online (Maybank)
- Kenanga Digital Investing

**Full-Service Brokers**:
- Maybank Investment Bank
- CIMB Securities
- Public Investment Bank

### Books (Highly Recommended)

1. **Technical Analysis**:
   - "Technical Analysis of the Financial Markets" - John Murphy
   - "Japanese Candlestick Charting Techniques" - Steve Nison

2. **Trading Psychology**:
   - "Trading in the Zone" - Mark Douglas
   - "The Psychology of Trading" - Brett Steenbarger

3. **Risk Management**:
   - "The New Trading for a Living" - Alexander Elder
   - "Trade Your Way to Financial Freedom" - Van K. Tharp

### Community & Forums

- I3investor Forums: https://klse.i3investor.com/
- Reddit r/MalaysianStocks
- TradingView Community

**Remember**: Online forums can be helpful but always verify information independently!